In [2]:
from sqlalchemy import select, update, and_, cast, func, Integer, insert
from app.services.utils import *

In [7]:
with session_factory() as session:
    query = select(SalesReportOrm.__table__.columns).filter_by(id=1)
    # r = session.get(SalesReportOrm, ident=1)  # Способ получения сущности, работает только для одной записи
    r = session.execute(query).mappings().first()
    # Метод scalars() используется, когда мы ожидаем получить одну колонку результата, а не несколько полей. 
    # Например, когда мы запрашиваем всю модель (в нашем случае SalesReportOrm) или одно конкретное поле (например, SalesReportOrm).
    # r = session.execute(query).scalar()  # работает как scalars, только он вернет первую сущность как first()
    print('='*100)
    print(r)
    print('='*100)

2025-08-04 15:12:05,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-04 15:12:05,097 INFO sqlalchemy.engine.Engine SELECT sales_report.id, sales_report.user_id, sales_report.period_from, sales_report.period_to, sales_report.total_orders, sales_report.total_revenue, sales_report.avg_order_value, sales_report.top_product_name, sales_report.top_product_sales, sales_report.cancelled_orders, sales_report.successful_orders, sales_report.returned_orders, sales_report.payment_method_stats, sales_report.category_breakdown, sales_report.sales_by_day, sales_report.comment, sales_report.created_at 
FROM sales_report 
WHERE sales_report.id = %(id_1)s::INTEGER
2025-08-04 15:12:05,098 INFO sqlalchemy.engine.Engine [cached since 818.6s ago] [SQL parameters hidden due to hide_parameters=True]
{'id': 1, 'user_id': 1, 'period_from': datetime.date(2024, 8, 10), 'period_to': datetime.date(2024, 9, 5), 'total_orders': 27, 'total_revenue': 106274.17, 'avg_order_value': 3936.08, 'top_product_name': 

In [ ]:
with session_factory() as session:
    query = insert(UserOrm)
    # r = session.get(SalesReportOrm, ident=1)  # Способ получения сущности, работает только для одной записи
    r = session.execute(
        query, [UserOrm(name='Пользователь1').to_dict(), UserOrm(name='Пользователь2').to_dict(), UserOrm(name='Пользователь3').to_dict()]
    )
    session.commit()
    # Метод scalars() используется, когда мы ожидаем получить одну колонку результата, а не несколько полей. 
    # Например, когда мы запрашиваем всю модель (в нашем случае SalesReportOrm) или одно конкретное поле (например, SalesReportOrm).
    # r = session.execute(query).scalar()  # работает как scalars, только он вернет первую сущность как first()
    print('='*100)
    print(r)
    print('='*100)

In [ ]:
with session_factory() as session:
    users = session.scalars(
        insert(UserOrm).returning(UserOrm),
        [
            {"name": "spongebob", "fullname": "Spongebob Squarepants"},
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
            {"name": "squidward", "fullname": "Squidward Tentacles"},
            {"name": "ehkrabs", "fullname": "Eugene H. Krabs"},
        ],
    )
    session.commit()
    print(users.all())

2025-07-30 22:39:49,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-30 22:39:49,155 INFO sqlalchemy.engine.Engine INSERT INTO "user" (name, fullname) VALUES (%(name__0)s::VARCHAR, %(fullname__0)s::VARCHAR), (%(name__1)s::VARCHAR, %(fullname__1)s::VARCHAR), (%(name__2)s::VARCHAR, %(fullname__2)s::VARCHAR), (%(name__3)s::VARCHAR, %(fullname__3)s::VARCHAR), (%(name__4)s::VARCHAR, %(fullname__4)s::VARCHAR) RETURNING "user".id, "user".name, "user".fullname, "user".created_at, "user".updated_at
2025-07-30 22:39:49,157 INFO sqlalchemy.engine.Engine [cached since 42.2s ago (insertmanyvalues) 1/1 (unordered)] [SQL parameters hidden due to hide_parameters=True]
2025-07-30 22:39:49,161 INFO sqlalchemy.engine.Engine COMMIT
[User(id=20, name='spongebob', fullname='Spongebob Squarepants'), User(id=21, name='sandy', fullname='Sandy Cheeks'), User(id=22, name='patrick', fullname='Patrick Star'), User(id=23, name='squidward', fullname='Squidward Tentacles'), User(id=24, name='ehkrabs', fulln

In [ ]:
def education_examples():
    with session_factory() as session:
        user = session.get(UserOrm, 3)  # При обновлении через get будет 2 запроса, на получение и обновление
        user.name=random.choice(['Бобер', 'Цыпа', 'Голубио'])
        user.fullname=random.choice(['Бобер', 'Цыпа', 'Голубио'])
        # session.flush()  сохраняет изменения в бд, но не завершает сессию как commit
        session.expire(user)  # сброс незакоммиченных изменений объекта user
        session.expire_all()  # сброс всех незакоммиченных изменений в сессии
        print(user.fullname)  # после expire\expire_all при попытке доступа к полям объекта будет выполнен новый запрос к БД
        session.refresh(user)  # делает запрос в бд на получение актуальных значений

In [ ]:
from app.users.dao import UserDAO
import random

# id: dict = UserDAO.create(username=f"spongebob{round(random.random(), 3)}", fullname="Spongebob Squarepants", password='acuna_matata')
# UserDAO.create_bulk([{'username': "spongebo221b", 'fullname':"Spongebob Squarepants"}, {'username': "spong11ebo1b", 'fullname':"Spongebob Squarepants"}])
# UserDAO.delete(id=69)
print('='*100)
user = UserDAO.find_one_or_none(id=3)
user_pass = user['password']
print(user_pass)
UserDAO.is_valid_password('acuna_matata', user_pass)

2025-08-09 18:51:19,393 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-09 18:51:19,394 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".username, "user".password, "user".fullname, "user".email, "user".disabled, "user".created_at, "user".updated_at 
FROM "user" 
WHERE "user".id = %(id_1)s::INTEGER
2025-08-09 18:51:19,394 INFO sqlalchemy.engine.Engine [cached since 2.938s ago] [SQL parameters hidden due to hide_parameters=True]
2025-08-09 18:51:19,396 INFO sqlalchemy.engine.Engine ROLLBACK
$2b$12$gwGmvdhGNO2FW1pqqSQpOOgQSmX5jHpGM2Zv8TeK4shy9kFm0V8ui


True

In [ ]:
import bcrypt
password = '$2b$12$gwGmvdhGNO2FW1pqqSQpOOgQSmX5jHpGM2Zv8TeK4shy9kFm0V8ui'.encode()
# Hash a password for the first time, with a randomly-generated salt
hashed = bcrypt.hashpw(password, bcrypt.gensalt())
print(hashed)
# Check that an unhashed password matches one that has previously been
# hashed
if bcrypt.checkpw(password, hashed):
    print("It Matches!")
else:
    print("It Does not Match :(")

b'$2b$12$zwb/ZwhPiwc1Fayz3KtcMuPpxjSmN3TmtlU5MjspCroFSH2sdrRxC'
It Matches!


In [ ]:
import random
from app.users.dao import UserDAO


ids: dict = UserDAO.create_bulk([
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
])
print('='*100)
print(ids)
for id in ids:
    user = UserDAO.find_one_or_none(id=id['id'])
    user_pass = user['password']
    print('='*100)
    print(user_pass)
    print(UserDAO.is_valid_password('acuna_matata', user_pass))

2025-08-09 18:56:10,824 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-09 18:56:10,826 INFO sqlalchemy.engine.Engine INSERT INTO "user" (username, password, fullname, disabled) VALUES (%(username_m0)s::VARCHAR, %(password_m0)s::VARCHAR, %(fullname_m0)s::VARCHAR, %(disabled)s), (%(username_m1)s::VARCHAR, %(password_m1)s::VARCHAR, %(fullname_m1)s::VARCHAR, %(disabled_m1)s), (%(username_m2)s::VARCHAR, %(password_m2)s::VARCHAR, %(fullname_m2)s::VARCHAR, %(disabled_m2)s), (%(username_m3)s::VARCHAR, %(password_m3)s::VARCHAR, %(fullname_m3)s::VARCHAR, %(disabled_m3)s) RETURNING "user".id
2025-08-09 18:56:10,826 INFO sqlalchemy.engine.Engine [no key 0.00058s] [SQL parameters hidden due to hide_parameters=True]
2025-08-09 18:56:10,827 INFO sqlalchemy.engine.Engine COMMIT
[{'id': 12}, {'id': 13}, {'id': 14}, {'id': 15}]
2025-08-09 18:56:10,836 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-09 18:56:10,836 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".username, "user".pas